# Determining Consumer Spend

Now that we have our demographics split out in the previous notebook we can focus on trying to solve our core problem of predicting consumer spend. 

To perform the prediction I will use a supervised learn technique called xgboost. This method has been used very successfully recently in both industry and by kaggle data scientist to solve some complex problems. It has become popular due to its high accuracy and low computation costs but the downside is that it has a lot of Hyper parameters to train. 

XGboost is a decision tree algorithm meaning that it splits data based on information gain (entropy) in order to group values. Normal decision tree methods (such as random forests) use a bagging which means many trees are produced and grouped together in an attempt to reduce overfitting.

Similarly to the k-means modeling first I will perform some preprocessing this time to combine our demographics data with the spend by day data. Then I will split the data into testing, training and validation datasets in order to fully test our model. The model will aim to predict spend for the demographic as a whole as this should be easier than individuals at this point. The prediction will be based on the number of consumers, the offers they received, demographic and other factors that may become apparent throughout the analysis. 

### Imports

In [1]:
# Install xgboost if needed using the line of code below
# ! pip install xgboost

In [144]:
# import general functions
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt

# import functions for modelling
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

# import the cleaning package
import sbpkg as sb

### Functions

### Global Variables

In [3]:
# read in the different datasources
portfolio_df = pd.read_json('data/portfolio.json', lines=True)
profile_df = pd.read_json('data/profile.json', lines=True)
transcript_df = pd.read_json('data/transcript.json', lines=True)

### Run Cleaning Functions

In [12]:
# run the initial cleaning on each dataset
clean_port_df = sb.clean_portfolio_data(portfolio_df)
clean_prof_df = sb.clean_profile_data(profile_df)
clean_trans_df = sb.clean_transcript_data(transcript_df)

# calculates the uninfluenced transactions for the modeling
uninflunced_trans = sb.norm_transactions(clean_trans_df, clean_port_df)

# process the user data to create the modeling input
user_data = sb.user_transactions(clean_prof_df, uninflunced_trans)

# load in the user spend by day
spd = sb.spend_per_day(clean_trans_df, clean_port_df)
spd.reset_index(inplace=True)

# predict the demographic of all the users in the data
predictions = sb.predict_demographic(user_data)
predictions.head()

,female,male,other,unknown gender,age,member joined,person,income,total transactions,total spend,spend per trans,spend per day,membership length,demographic
0,0,0,0,1,0,2017-02-12,68be06ca386d4c31939f3a4f0e3dd783,0.0,9,20.40,2.266667,0.680000,76.0,1
1,1,0,0,0,55,2017-07-15,0610b486422d4921ae7d2bf64640c50b,112000.0,3,77.01,25.670000,2.567000,54.0,7
2,0,0,0,1,0,2018-07-12,38fe809add3b4fcf9315a9694bb96ff5,0.0,5,10.21,2.042000,0.340333,2.0,1
3,1,0,0,0,75,2017-05-09,78afa995795e4d85b5d9ceeca43f5fef,100000.0,4,89.99,22.497500,2.999667,63.0,7
4,0,0,0,1,0,2017-08-04,a03223e636434f42ac4c3df47e8bac43,0.0,3,4.65,1.550000,0.155000,51.0,1


### Data Preparation

In [42]:
# drop everything except the demographic from the prediction data
demographics = predictions[['person','demographic']]

# merge the two datasets so that we have the demographic data for each person
input_data = spd.merge(demographics, on=['person'])
input_data.head()

# only keep the first 23 days so that the last 7 can be used for modeling
input_data = input_data[input_data['transaction_time'] < 24]

# sum the spend & number of offers that 
input_data = input_data.groupby(['transaction_time','person']).sum()
input_data.tail()

spend  \
transaction_time person                                     
23.0             fff29fb549084123bd046dbc5ceb4faa   32.57   
                 fff3ba4757bd42088c044ca26d73817a  414.31   
                 fff7576017104bcc8677a8d63322b5e1    0.00   
                 fffad4f4828548d1b5583907f2e9906b    0.00   
                 ffff82501cea40309d5fdd7edcca4a07    0.00   

                                                   0b1e1539f2cc45b7b9fa7c272da2e1d7  \
transaction_time person                                                               
23.0             fff29fb549084123bd046dbc5ceb4faa                               0.0   
                 fff3ba4757bd42088c044ca26d73817a                               0.0   
                 fff7576017104bcc8677a8d63322b5e1                               0.0   
                 fffad4f4828548d1b5583907f2e9906b                               0.0   
                 ffff82501cea40309d5fdd7edcca4a07                               0.0   

                                                   2298d6c36e964ae4a3e7e9706d1fb8c2  \
transaction_time person                                                               
23.0             fff29fb549084123bd046dbc5ceb4faa                               0.0   
                 fff3ba4757bd42088c044ca26d73817a                               0.0   
                 fff7576017104bcc8677a8d63322b5e1                               0.0   
                 fffad4f4828548d1b5583907f2e9906b                               0.0   
                 ffff82501cea40309d5fdd7edcca4a07                               0.0   

                                                   2906b810c7d4411798c6938adc9daaa5  \
transaction_time person                                                               
23.0             fff29fb549084123bd046dbc5ceb4faa                               0.0   
                 fff3ba4757bd42088c044ca26d73817a                               0.0   
                 fff7576017104bcc8677a8d63322b5e1                               0.0   
                 fffad4f4828548d1b5583907f2e9906b                               0.0   
                 ffff82501cea40309d5fdd7edcca4a07                               0.0   

                                                   3f207df678b143eea3cee63160fa8bed  \
transaction_time person                                                               
23.0             fff29fb549084123bd046dbc5ceb4faa                               0.0   
                 fff3ba4757bd42088c044ca26d73817a                               0.0   
                 fff7576017104bcc8677a8d63322b5e1                               0.0   
                 fffad4f4828548d1b5583907f2e9906b                               0.0   
                 ffff82501cea40309d5fdd7edcca4a07                               0.0   

                                                   4d5c57ea9a6940dd891ad53e9dbe8da0  \
transaction_time person                                                               
23.0             fff29fb549084123bd046dbc5ceb4faa                               0.0   
                 fff3ba4757bd42088c044ca26d73817a                               0.0   
                 fff7576017104bcc8677a8d63322b5e1                               0.0   
                 fffad4f4828548d1b5583907f2e9906b                               0.0   
                 ffff82501cea40309d5fdd7edcca4a07                               0.0   

                                                   5a8bc65990b245e5a138643cd4eb9837  \
transaction_time person                                                               
23.0             fff29fb549084123bd046dbc5ceb4faa                               0.0   
                 fff3ba4757bd42088c044ca26d73817a                               2.0   
                 fff7576017104bcc8677a8d63322b5e1                               0.0   
                 fffad4f4828548d1b5583907f2e9906b                               0.0   
                 ffff82501cea40309d5fdd7edcca4a07   

In [52]:
# To perform testing on the data I will only keep one demographic for now
input_data_test = input_data[input_data['demographic'] == 0]

# reset the index
input_data_test.reset_index(inplace=True)

# # keep only the columns needed
input_data_test = input_data_test.drop(columns=['transaction_time','person','demographic'])
input_data_test.head()

,spend,0b1e1539f2cc45b7b9fa7c272da2e1d7,2298d6c36e964ae4a3e7e9706d1fb8c2,2906b810c7d4411798c6938adc9daaa5,3f207df678b143eea3cee63160fa8bed,4d5c57ea9a6940dd891ad53e9dbe8da0,5a8bc65990b245e5a138643cd4eb9837,9b98b8c7a33c4b65b9aebfe6a799e6d9,ae264e3637204a6fb9bb56bc8210ddfd,f19421c1d4aa40978ebb69ca19b0e20d,fafdcd668e3743c1bb461111dcafc2a4
0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,25.62,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# split the data again into testing and training data
X_train, X_test, y_train, y_test = train_test_split(input_data_test.iloc[:,1:],
                                                   input_data_test.iloc[:,0],
                                                   test_size=.1, 
                                                   random_state=42)

# change the input to a dmatrix to improve optimsation speed for testing
# Note DMatrix was choosen to replace the standard xgboost to reduce the training times
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# calculate the baseline RMSE error, this will be used for evaluation of the model
baseline_pred = np.ones(y_test.shape) * np.mean(y_train)
mae_baseline = np.sqrt(mean_squared_error(y_test, baseline_pred))

print(f"Baseline RMSE is {mae_baseline}")

### Modelling Spend

To start I will set up the initial parameters to x.

In [182]:
# set up the intial parameters for the xgboosting parameterisation
params = {
    'max_depth':5,
    'min_child_weight': 1,
    'eta':.1,
    'subsample': 1,
    'colsample_bytree': 1,
    'gamma':0.1,
    'objective':'reg:squarederror',
}

First, I will set the x & y values due to z.

In [197]:
# set the rmse value as a very high value
rmse = 1000

# set_ best parameters as nothing
best_parms = None

# sets values for max_depth & min_child_weight
testing_prams = [
    (max_depth, min_child_weight)
    for max_depth in range(7,12)
    for min_child_weight in range(3,8)
]

# loop through both of the variablles
for max_depth, min_child_weight in testing_prams:
    # update the parameters
    params['max_depth'] = max_depth
    params['min_child_weight'] = min_child_weight
    
    # Run CV
    results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=14,
        nfold=5,
        metrics={'rmse'},
        early_stopping_rounds=10
    )
    
    # save the best rmse score
    best_rmse = results['test-rmse-mean'].min()
    
    # print out the results
    print(f"max_depth = {max_depth} | "
          f"min_child_weight = {min_child_weight} | "
          f"rmse = {best_rmse}")
    
    # set the best parameters if they improve on the current rmse
    if best_rmse < rmse:
        rmse = best_rmse
        best_params = (max_depth, min_child_weight)
        
print(f"Best params: "
      f"max_depth = {best_params[0]} | "
      f"min_child_weight = {best_params[1]} | "
      f"rmse = {rmse}")

max_depth = 7 | min_child_weight = 3 | rmse = 9.1717962
max_depth = 7 | min_child_weight = 4 | rmse = 9.1715834
max_depth = 7 | min_child_weight = 5 | rmse = 9.1721596
max_depth = 7 | min_child_weight = 6 | rmse = 9.1729186
max_depth = 7 | min_child_weight = 7 | rmse = 9.1727892
max_depth = 8 | min_child_weight = 3 | rmse = 9.171504
max_depth = 8 | min_child_weight = 4 | rmse = 9.1711604
max_depth = 8 | min_child_weight = 5 | rmse = 9.171302
max_depth = 8 | min_child_weight = 6 | rmse = 9.1714588
max_depth = 8 | min_child_weight = 7 | rmse = 9.171916
max_depth = 9 | min_child_weight = 3 | rmse = 9.1723122
max_depth = 9 | min_child_weight = 4 | rmse = 9.1717366
max_depth = 9 | min_child_weight = 5 | rmse = 9.1718656
max_depth = 9 | min_child_weight = 6 | rmse = 9.1719034
max_depth = 9 | min_child_weight = 7 | rmse = 9.1718762
max_depth = 10 | min_child_weight = 3 | rmse = 9.173150600000001
max_depth = 10 | min_child_weight = 4 | rmse = 9.172769599999999
max_depth = 10 | min_child_weight

In [198]:
# set the new best parameters based on the testing
params['max_depth'] = 8
params['min_child_weight'] = 4

Now I will set the values of y & x due to z.

In [205]:
# set the rmse value as a very high value
rmse = 1000

# set_ best parameters as nothing
best_parms = None

# sets values for max_depth & min_child_weight
testing_prams = [
    (subsample, colsample_bytree)
    for subsample in [i/10. for i in range(5,11)]
    for colsample_bytree in [i/10. for i in range(5,11)]
]

# loop through both of the variablles
for subsample, colsample_bytree in testing_prams:
    # update the parameters
    params['subsample'] = subsample
    params['colsample_bytree'] = colsample_bytree
    
    # Run CV
    results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=14,
        nfold=5,
        metrics={'rmse'},
        early_stopping_rounds=10
    )
    
    # save the best rmse score
    best_rmse = results['test-rmse-mean'].min()
    
    # print out the results
    print(f"subsample = {subsample} | "
          f"colsample_bytree = {colsample_bytree} | "
          f"rmse = {best_rmse}")
    
    # set the best parameters if they improve on the current rmse
    if best_rmse < rmse:
        rmse = best_rmse
        best_params = (subsample, colsample_bytree)
        
print(f"Best params: "
      f"subsample = {best_params[0]} | "
      f"colsample_bytree = {best_params[1]} | "
      f"rmse = {rmse}")

subsample = 0.5 | colsample_bytree = 0.5 | rmse = 9.182648599999998
subsample = 0.5 | colsample_bytree = 0.6 | rmse = 9.167146599999999
subsample = 0.5 | colsample_bytree = 0.7 | rmse = 9.1572024
subsample = 0.5 | colsample_bytree = 0.8 | rmse = 9.154855799999998
subsample = 0.5 | colsample_bytree = 0.9 | rmse = 9.1543384
subsample = 0.5 | colsample_bytree = 1.0 | rmse = 9.160515799999999
subsample = 0.6 | colsample_bytree = 0.5 | rmse = 9.1864538
subsample = 0.6 | colsample_bytree = 0.6 | rmse = 9.171585799999999
subsample = 0.6 | colsample_bytree = 0.7 | rmse = 9.1602526
subsample = 0.6 | colsample_bytree = 0.8 | rmse = 9.1579808
subsample = 0.6 | colsample_bytree = 0.9 | rmse = 9.1573866
subsample = 0.6 | colsample_bytree = 1.0 | rmse = 9.1620508
subsample = 0.7 | colsample_bytree = 0.5 | rmse = 9.1923356
subsample = 0.7 | colsample_bytree = 0.6 | rmse = 9.176516000000001
subsample = 0.7 | colsample_bytree = 0.7 | rmse = 9.163980200000001
subsample = 0.7 | colsample_bytree = 0.8 | r

In [206]:
params['subsample'] = 0.5
params['colsample_bytree'] = 0.9

xxxxx

In [ ]:
# set the rmse value as a very high value
rmse = 1000

# set_ best parameters as nothing
best_parms = None

# loop through both of the variablles
for gamma in [.3, .2, .1, .05, .01, .005]:
    # update the parameters
    params['gamma'] = gamma
    
    # Run CV
    results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=14,
        nfold=5,
        metrics={'rmse'},
        early_stopping_rounds=10
    )
    
    # save the best rmse score
    best_rmse = results['test-rmse-mean'].min()
    
    # print out the results
    print(f"gamma = {subsample} | "
          f"rmse = {best_rmse}")
    
    # set the best parameters if they improve on the current rmse
    if best_rmse < rmse:
        rmse = best_rmse
        best_params = (subsample, colsample_bytree)
        
print(f"Best params: "
      f"gamma = {best_params[0]} | "
      f"rmse = {rmse}")

In [ ]:
params['gamma'] = 0.1

Finally I will set the values of x & y to determine z.

In [213]:
# set the rmse value as a very high value
rmse = 1000

# set_ best parameters as nothing
best_parms = None

# loop through both of the variablles
for eta in [.3, .2, .1, .05, .01, .005]:
    # update the parameters
    params['eta'] = eta
    
    # Run CV
    results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=14,
        nfold=5,
        metrics={'rmse'},
        early_stopping_rounds=10
    )
    
    # save the best rmse score
    best_rmse = results['test-rmse-mean'].min()
    
    # print out the results
    print(f"eta = {eta} | "
          f"rmse = {best_rmse}")
    
    # set the best parameters if they improve on the current rmse
    if best_rmse < rmse:
        rmse = best_rmse
        best_params = (eta)
        
print(f"Best params: "
      f"eta = {best_params} | "
      f"rmse = {rmse}")

eta = 0.3 | rmse = 9.185653399999998
eta = 0.2 | rmse = 9.163082999999999
eta = 0.1 | rmse = 9.1543384
eta = 0.05 | rmse = 9.1589534
eta = 0.01 | rmse = 9.1550306
eta = 0.005 | rmse = 9.156702800000001
Best params: eta = 0.1 | rmse = 9.1543384


In [210]:
params['eta'] = .1

In [211]:
# test the number of boosted rounds needed to get the best model
best_model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")]
)

# test the predictions created above
pred = best_model.predict(dtest)

# 
rmse_new = np.sqrt(mean_squared_error(pred, y_test))

[0]	Test-rmse:9.14369
[1]	Test-rmse:8.95792
[2]	Test-rmse:8.81633
[3]	Test-rmse:8.70471
[4]	Test-rmse:8.60015
[5]	Test-rmse:8.51371
[6]	Test-rmse:8.44556
[7]	Test-rmse:8.39372
[8]	Test-rmse:8.34461
[9]	Test-rmse:8.3038
[10]	Test-rmse:8.27654
[11]	Test-rmse:8.24978
[12]	Test-rmse:8.23128
[13]	Test-rmse:8.21453
[14]	Test-rmse:8.20128
[15]	Test-rmse:8.19466
[16]	Test-rmse:8.18623
[17]	Test-rmse:8.17936
[18]	Test-rmse:8.17684
[19]	Test-rmse:8.17607
[20]	Test-rmse:8.17314
[21]	Test-rmse:8.17425
[22]	Test-rmse:8.1739
[23]	Test-rmse:8.16995
[24]	Test-rmse:8.17201
[25]	Test-rmse:8.17251
[26]	Test-rmse:8.16945
[27]	Test-rmse:8.16838
[28]	Test-rmse:8.16543
[29]	Test-rmse:8.16844
[30]	Test-rmse:8.16964
[31]	Test-rmse:8.17137
[32]	Test-rmse:8.17696
[33]	Test-rmse:8.17322
[34]	Test-rmse:8.17468
[35]	Test-rmse:8.16883
[36]	Test-rmse:8.16804
[37]	Test-rmse:8.17087
[38]	Test-rmse:8.16955
[39]	Test-rmse:8.16981
[40]	Test-rmse:8.16955
[41]	Test-rmse:8.17005
[42]	Test-rmse:8.17127
[43]	Test-rmse:8.17595


[348]	Test-rmse:8.16869
[349]	Test-rmse:8.16699
[350]	Test-rmse:8.1663
[351]	Test-rmse:8.1673
[352]	Test-rmse:8.16784
[353]	Test-rmse:8.16702
[354]	Test-rmse:8.16668
[355]	Test-rmse:8.17114
[356]	Test-rmse:8.17106
[357]	Test-rmse:8.17053
[358]	Test-rmse:8.16857
[359]	Test-rmse:8.16697
[360]	Test-rmse:8.16888
[361]	Test-rmse:8.17363
[362]	Test-rmse:8.17421
[363]	Test-rmse:8.17098
[364]	Test-rmse:8.17318
[365]	Test-rmse:8.1741
[366]	Test-rmse:8.16737
[367]	Test-rmse:8.16714
[368]	Test-rmse:8.16862
[369]	Test-rmse:8.16937
[370]	Test-rmse:8.16904
[371]	Test-rmse:8.16872
[372]	Test-rmse:8.16755
[373]	Test-rmse:8.16603
[374]	Test-rmse:8.1692
[375]	Test-rmse:8.16898
[376]	Test-rmse:8.16652
[377]	Test-rmse:8.16776
[378]	Test-rmse:8.17004
[379]	Test-rmse:8.17172
[380]	Test-rmse:8.17224
[381]	Test-rmse:8.17212
[382]	Test-rmse:8.17473
[383]	Test-rmse:8.17545
[384]	Test-rmse:8.17863
[385]	Test-rmse:8.17926
[386]	Test-rmse:8.17911
[387]	Test-rmse:8.18327
[388]	Test-rmse:8.18429
[389]	Test-rmse:8.18

[691]	Test-rmse:8.16553
[692]	Test-rmse:8.16567
[693]	Test-rmse:8.16377
[694]	Test-rmse:8.16461
[695]	Test-rmse:8.16995
[696]	Test-rmse:8.16774
[697]	Test-rmse:8.1707
[698]	Test-rmse:8.1705
[699]	Test-rmse:8.16797
[700]	Test-rmse:8.16613
[701]	Test-rmse:8.16494
[702]	Test-rmse:8.16672
[703]	Test-rmse:8.17025
[704]	Test-rmse:8.16905
[705]	Test-rmse:8.16881
[706]	Test-rmse:8.16815
[707]	Test-rmse:8.16985
[708]	Test-rmse:8.17036
[709]	Test-rmse:8.1728
[710]	Test-rmse:8.17056
[711]	Test-rmse:8.17304
[712]	Test-rmse:8.1712
[713]	Test-rmse:8.17257
[714]	Test-rmse:8.17178
[715]	Test-rmse:8.17473
[716]	Test-rmse:8.17296
[717]	Test-rmse:8.17483
[718]	Test-rmse:8.17564
[719]	Test-rmse:8.18192
[720]	Test-rmse:8.18158
[721]	Test-rmse:8.17861
[722]	Test-rmse:8.17807
[723]	Test-rmse:8.17275
[724]	Test-rmse:8.16973
[725]	Test-rmse:8.1724
[726]	Test-rmse:8.16833
[727]	Test-rmse:8.16923
[728]	Test-rmse:8.17083
[729]	Test-rmse:8.17173
[730]	Test-rmse:8.1701
[731]	Test-rmse:8.17174
[732]	Test-rmse:8.1681

In [214]:
rmse_new

8.17104533078391

Now that I have the optimum parameters for a model I can loop through each of the demographics and create a model for each. This should predict 

In [179]:
def create_spend_model(spend_data, demographics_data, model_demographic):
    """
    this function is used create a model that can predict the price for a given demographic
    """
    # drop everything except the demographic from the prediction data
    demographics = predictions[['person','demographic']]

    # merge the two datasets so that we have the demographic data for each person
    input_data = spd.merge(demographics, on=['person'])
    input_data.head()

    # only keep the first 23 days so that the last 7 can be used for modeling
    input_data = input_data[input_data['transaction_time'] < 24]

    # sum the spend & number of offers that are influenced 
    input_data = input_data.groupby(['transaction_time','person']).sum()
    
    # To perform testing on the data I will only keep one demographic for now
    input_data_test = input_data[input_data['demographic'] == model_demographic]

    # reset the index
    input_data_test.reset_index(inplace=True)

    # keep only the columns needed for modeling
    input_data_test = input_data_test.drop(columns=['transaction_time','person','demographic'])
    
    # split the data again into testing and training data
    X_train, X_test, y_train, y_test = train_test_split(input_data_test.iloc[:,1:],
                                                        input_data_test.iloc[:,0],
                                                        test_size=.1, 
                                                        random_state=14)
    
    
    # creates model using the XGB classifier    
    model = xgb.XGBRegressor(max_depth=10,
                min_child_weight=7.0,
                subsample=1.0,
                colsample_bytree=1.0,
                objective='reg:squarederror',
                n_estimators=999,
                learning_rate=0.01)
    
    # fit the model to the training data    
    model.fit(X_train, y_train)
    
    # calculate the rmse error
    test_pred = model.predict(X_test)
    mod_rmse = mean_absolute_error(test_pred, y_test)
    
    # calculate baseline rmse
    mean_train = np.mean(y_train)
    baseline_pred = np.ones(y_test.shape) * mean_train
    base_rmse = mean_absolute_error(y_test, baseline_pred)
    
    # save the model ready for the predictions
    model.save_model(f"xgboost_price_model_{model_demographic}.model")
    
    # print rmse compaired to the base level
    print(f"Baseline RMSE is {str(base_rmse)}")
    print(f"Model RMSE: {str(mod_rmse)}")   

In [181]:
# create the model for each of the different demographics:
for demographic in [0,1,2,3,4,6,7]:
    create_spend_model(spd, predictions, demographic)

/Users/warwick.rommelrath/.pyenv/versions/3.7.5/envs/datastuff/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


Baseline RMSE is 5.001785682809074
Model RMSE: 4.015500261796952


In [ ]:
def predict_spend(data, model_demographic):
    """
    this function predicts the spend of users based on the model made for there demographic
    """
    #     

### Modelling Analysis

In [ ]:
demographics_output.groupby('cluster').mean()

In [ ]:
demographics_output.groupby('cluster')['age'].describe()

In [121]:
best_model.eval(dtrain)

'[0]\teval-rmse:9.269566'